# Download file:

In [ ]:
# !pip install pyspellchecker rapidfuzz
# !pip freeze > ../requirements.txt

     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
     - -------------------------------------- 0.2/7.2 MB 3.7 MB/s eta 0:00:02
     -- ------------------------------------- 0.5/7.2 MB 5.6 MB/s eta 0:00:02
     --- ------------------------------------ 0.6/7.2 MB 5.0 MB/s eta 0:00:02
     ---- ----------------------------------- 0.8/7.2 MB 5.1 MB/s eta 0:00:02
     ----- ---------------------------------- 1.0/7.2 MB 4.9 MB/s eta 0:00:02
     ------ --------------------------------- 1.3/7.2 MB 5.0 MB/s eta 0:00:02
     -------- ------------------------------- 1.6/7.2 MB 5.2 MB/s eta 0:00:02
     --------- ------------------------------ 1.8/7.2 MB 5.2 MB/s eta 0:00:02
     ----------- ---------------------------- 2.1/7.2 MB 5.5 MB/s eta 0:00:01
     ------------- -------------------------- 2.4/7.2 MB 5.5 MB/s eta 0:00:01
     -------------- ------------------------- 2.7/7.2 MB 5.5 MB/s eta 0:00:01
     ---------------- ----------------------- 2.9/7.2 MB 5.6 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import shutil, os
import pandas as pd

In [4]:

# Reset method: remove the entire downloaded recipe-dataset-over-2m directory to redownload from scratch
def reset_kagglehub_dataset(dataset_name):
    import shutil, os
    cache_dir = os.path.expanduser("~/.cache/kagglehub/datasets")
    dataset_dir = os.path.join(cache_dir, dataset_name.replace('/', os.sep))
    if os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)
        print(f"Removed dataset directory: {dataset_dir}")
    else:
        print(f"Dataset directory not found: {dataset_dir}")

# Download and move a file from Kaggle dataset to a target path, with option to reset cache first
def download_and_move_kaggle_file(file_path, kaggle_dataset, kaggle_filename="recipes_data.csv", replace=True):
    import kagglehub, os, shutil
    # If replace is True, reset KaggleHub cache for this dataset
    if replace:
        reset_kagglehub_dataset(kaggle_dataset)
    # Download dataset
    kaggle_dir = kagglehub.dataset_download(kaggle_dataset)
    print(f"Downloaded dataset to: {kaggle_dir}")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    src_file = os.path.join(kaggle_dir, kaggle_filename)
    if not os.path.exists(src_file):
        print(f"{kaggle_filename} not found in {kaggle_dir}.")
        return False
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Existing file {file_path} removed.")
    shutil.move(src_file, file_path)
    print(f"Moved {kaggle_filename} to {file_path}.")
    return True

In [ ]:
paths = [
    # ("alaknandaa/recipes-data-by-cuisine", "./all_cuisines.xlsx"),
    # ("wilmerarltstrmberg/recipe-dataset-over-2m", "recipes_data.csv"),
    # ("snehallokesh31096/recipe", "recipes_82k.csv"),
    # ("mfarazf/cuisine-dataset", "New file.csv"),
    # ("sarthak71/food-recipes", "food_recipes.csv"),
    ("ajitrajput/foodrecipes", "recipes.csv"),
]

for dataset, filename in paths:

    # create file path if not exists
    try:
        file_path = os.path.abspath(f"../data/{filename.split('.')[0]}_data.csv")
    except Exception as e:
        # add path[0] to file_path and try again
        file_path = os.path.abspath(f"../data/{dataset.split('/')[-1]}_{filename.split('.')[0]}.csv")
    print(f"✅ Loaded file: {file_path}")
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"⚠️ Could not load {file_path} as CSV: {e}")
        continue
    display(df.head())


Removed dataset directory: C:\Users\georg.DESKTOP-2FS9VF1/.cache/kagglehub/datasets\ajitrajput\foodrecipes


100%|██████████| 299k/299k [00:00<00:00, 1.48MB/s]

Extracting files...
Downloaded dataset to: C:\Users\georg.DESKTOP-2FS9VF1\.cache\kagglehub\datasets\ajitrajput\foodrecipes\versions\1


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\georg.DESKTOP-2FS9VF1\\source\\repos\\699-capstone-team14\\data\\recipes_data.csv'

## Composition

| Column Name   | Description                                                        |
|---------------|--------------------------------------------------------------------|
| title         | Title of the dish                                                  |
| ingredients   | The ingredients and amount (Unorganized)                           |
| directions    | The instructions (Unorganized)                                     |
| link          | Link to the dish                                                   |
| source        | Either gathered or imported from other dataset                     |
| NER           | Ingredients without amount, brand, etc. (Organized)                |
| site          | The domain from where it has been sourced                          |
